In [ ]:
# To avoid certain errors that occur in some systems due to conflicts of paths.
import os, sys, pprint
# Add the parent directory of export to the Python path
working_dir = os.getcwd()
parent_dir = os.path.join(os.getcwd(), os.pardir)
p_parent_dir = os.path.join(parent_dir, os.pardir) 
sys.path.append(os.path.abspath(p_parent_dir))

In [ ]:
#All the import statements required in the generation of lci
import bw2data as bw
import bw2io as bwio
import shutil as sh
#import json
import pandas as pd
import export.libs.metadata as md
import export.libs.excel as excel
from network.libs import lci, extract

## Export the LCI database


### Initialize
Set the project and the required database

In [ ]:
project_name = input("Enter the name of the project: ")
database_name = input("Enter the name of the database: ")
bw.projects.set_current(project_name)
selected_database = bw.Database(database_name)
# All output files will contain the exported_file_name as the prefix
exported_file_name = input("Enter the desired name for the export files: ")

### Export to csv fle

In [ ]:
file_location = bwio.export.csv.write_lci_csv(database_name) #this excludes nested data
working_directory = os.getcwd()
print ('The current working directory is ', working_directory)
sh.copy(file_location, exported_file_name+'.csv' )

### Export to excel file

Create the excel lci inventory

In [ ]:
excel_file_location = bwio.export.excel.write_lci_excel(database_name)
working_directory = os.getcwd()
print ('The current working directory is ', working_directory)
sh.copy(excel_file_location, exported_file_name + '.xlsx' )

Format or "beautify" this excel file to make it more human-readable

In [ ]:
excel_file = input("Enter the path of your excel file: ")
output_excel_file = exported_file_name + '_formatted.xlsx'
excel.excel_format(excel_file, output_excel_file)

### Export the database properties

General properties of the database such as total no. of activities, exchanges, project parameters etc. At present, the database versioning has to performed manually. Methods of making this process a bit intelligent (Depending on the underlying changes in the data) has to be investigated separately.

In [ ]:
database_properties = md.db_props(project_name, database_name).gen_info()
import uuid
uid = uuid.uuid4().hex
database_properties['Database Identifier'] = uid
database_properties['Version'] = input ("Enter the version of the database (e.g 1.0.1): ")
pprint.pprint (database_properties)

Get the names of all the dependent databases within the selcted database. Append this to the database properties dict.

In [ ]:
lci_db = lci.LCI(project_name, database_name)

#declare the important info
keys_technosphere = ['name', 'database', 'location', 'unit','reference product']
keys_biosphere = ['name', 'database', 'unit', 'categories']
keys_production = ['name', 'database', 'location', 'unit','reference product']

# the life cycle inventory is stored as a list of dicts with first element of list containing parameters and second element containing activity dics.
LCI_list = lci_db.LCI_database_list(keys_biosphere, keys_technosphere, keys_production)
# pprint.pprint(LCI_list, indent=2)

Add the names of database dependencies to the database_properties dictionary

In [ ]:
dependencies = md.db_props.db_dependencies(LCI_list[1])
database_properties['Database Dependencies'] = dependencies
pprint.pprint (database_properties)

Write database_properties to a csv file

In [ ]:
db_props_csv_file = exported_file_name + '_properties.csv'
md.db_props.csv(db_props_csv_file, database_properties)

## Metadata file


### Metadata .md file from excel
If required, the excel template can be used to generate a markdown version. This is intended for anyone who may find it difficult to create or replicate markdown template based on /templates/datasets. *Note:* The xlsx to .md may not work always, expecially if there are too much formatting present in the source excel file.

In [ ]:
input_file_path = input("Enter the path to the Excel metadata file: ")         # Ask the user to provide the Excel file path
output_metadata_file = exported_file_name + '_metadata.md'

md.metadata().excel_to_md(input_file_path, output_metadata_file)